In [1]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v')

Source files will be saved in "/tmp/tmpq5ke7pkn".


We are so fucking back, have a sore throat, IM PISSED AND IN NEED TO CODE AND THINK IMMA GO TAKE SOME COUGHT SUPRESSSANT AND BE RIGHT BACK. 

In [ ]:
%%cuda

#include<stdio.h>
#include<stdlib.h>
#include<cuda.h> 
#include<cuda_runtime.h> 
#include<cooperative_groups.h> 


constexpr int N = 4096; 
constexpr int block_size = 1024; 
constexpr int N_blocks = 1; 
constexpr int N_iter = N/block_size;

__global__ void three_step_sync(float*A, float*B)
{
  
}

int main()
{
  float *A_h, *A_d, *B_h, *B_d; 
  size_t size = N*sizeof(float);
  
  cudaHostAlloc(&A_h, size, cudaHostAllocDefault);
  cudaHostAlloc(&B_h, size, cudaHostAllocDefault); 
  
  cudaMalloc(&A_d, size);
  cudaMalloc(&B_d, size);
  
  for (int i = 0; i < N; i++)
  {
    A_h[i] = (float)(i/1000.0);
  }
  
  cudaMemcpy(A_d, A_h, cudaMemcpyHostToDevice); 
  
  three_set_sync<<<N_blocks, block_size>>>(A_d, B_d); 
  
  cudaDeviceSyncronize();
  
  cudaMemcpy(B_h, B_d, cuddaMemcpyDeviceToHost);
  
  
  cudaFree(A_d)
  cudaFree(B_d) 
    
  
  
  
  
  
  
  
  
}
